# S03 E01: Handcrafted Feature Extraction
### Créditos: Jederson e Myllena

## Import Libraries 

In [1]:
import pandas as pd
import numpy as np
import librosa 
import os
import glob
import re
import pickle
from tqdm import tqdm

## Extract Features
### 13 features
Cepstral: MFCC | Time: ZCR, centroid, rolloff, tempogram | Frequency: chroma* 3, bandwidth, contrast, flatness, poly, tonnez

In [2]:
def extract_features_mfcc(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc = 40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("MFCC => Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccsscaled

In [3]:
def extract_features_zero(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        zcr = librosa.zero_crossings(audio, pad = False)
        zcr = np.sum(zcr)
        pass
        
    except Exception as e:
        print("ZCR => Error encountered while parsing file: ", file_name)
        return None 
     
    return zcr

In [4]:
def extract_features_chroma_stft(file_name):

  try: 
      audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
      ch_stft = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
      stft = np.mean(ch_stft)
      pass

  except Exception as e:
      print("STFT => Error encountered while parsing file: ", file_name)
      return None 
     
  return stft

In [5]:
def extract_features_chroma_cqt(file_name):

  try: 
      audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
      cqt = librosa.feature.chroma_cqt(y=audio, sr=sample_rate)
      cqt = np.mean(cqt)
      pass

  except Exception as e:
      print("CQT => Error encountered while parsing file: ", file_name)
      return None 
     
  return cqt

In [6]:
def extract_features_chroma_cens(file_name):

  try: 
      audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
      cens = librosa.feature.chroma_cens(y=audio, sr=sample_rate)
      cens = np.mean(cens)
      pass

  except Exception as e:
      print("Chroma cens => Error encountered while parsing file: ", file_name)
      return None 
     
  return cens

In [7]:
def extract_features_spectral_centroid(file_name):

  try: 
      audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
      cent = librosa.feature.spectral_centroid(y=audio, sr=sample_rate)
      cent = np.mean(cent)
      pass

  except Exception as e:
      print("Spectral centroid => Error encountered while parsing file: ", file_name)
      return None 
     
  return cent

In [8]:
def extract_features_spectral_bandwidth(file_name):

  try: 
      audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
      band_wd = librosa.feature.spectral_bandwidth(y=audio, sr=sample_rate)
      band_wd = np.mean(band_wd)
      pass

  except Exception as e:
      print("Spectral bandwith => Error encountered while parsing file: ", file_name)
      return None 
     
  return band_wd

In [9]:
def extract_features_spectral_contrast(file_name):

  try: 
      audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
      S = np.abs(librosa.stft(audio))
      contrast = librosa.feature.spectral_contrast(S=S, sr= sample_rate)
      contrast = np.mean(contrast, axis = 1)
      pass

  except Exception as e:
      print("Spectral contrast => Error encountered while parsing file: ", file_name)
      return None 
     
  return contrast

In [10]:
def extract_features_spectral_flatness(file_name):

  try: 
      audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
      flat = librosa.feature.spectral_flatness(y=audio)
      flat = np.mean(flat)
      pass

  except Exception as e:
      print("Spectral flatness => Error encountered while parsing file: ", file_name)
      return None 
     
  return flat

In [11]:
def extract_features_spectral_rolloff(file_name):

  try: 
      audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
      rolloff = librosa.feature.spectral_rolloff(y=audio, sr= sample_rate, roll_percent=0.85)
      #roll_percent padrão = 0.85, pode ser alterado
      rolloff = np.mean(rolloff)
      pass

  except Exception as e:
      print("Spectral roloff => Error encountered while parsing file: ", file_name)
      return None 
     
  return rolloff

In [12]:
def extract_features_poly_features(file_name):
  
  try: 
      audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
      S = np.abs(librosa.stft(audio))
      p0 = librosa.feature.poly_features(S=S, order=0)
      poly_f = np.mean(p0)
      pass

  except Exception as e:
      print("Poly features => Error encountered while parsing file: ", file_name)
      return None 
     
  return poly_f

In [13]:
def extract_features_tonnetz(file_name):
  
  try: 
      audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
      audio = librosa.effects.harmonic(audio)
      tonnetz = librosa.feature.tonnetz(y=audio, sr=sample_rate)
      tonnetz = np.mean(np.mean(tonnetz))
      pass

  except Exception as e:
      print("Tonnetz => Error encountered while parsing file: ", file_name)
      return None 
     
  return tonnetz

In [14]:
def extract_features_tempogram(file_name):
  
    try: 
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        hop_length = 385 #número de amostras de áudio entre medições sucessivas de início
        oenv = librosa.onset.onset_strength(y=audio, sr=sample_rate, hop_length=hop_length)
        tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sample_rate, hop_length=hop_length)
        tempogram = np.mean(tempogram)
        pass

    except Exception as e:
        print("Tempogram => Error encountered while parsing file: ", file_name)
        return None 
     
    return tempogram

In [15]:
features_function = []

features_function.append(extract_features_mfcc)
features_function.append(extract_features_zero)
features_function.append(extract_features_chroma_stft)
features_function.append(extract_features_chroma_cqt)
features_function.append(extract_features_chroma_cens)
features_function.append(extract_features_spectral_centroid)
features_function.append(extract_features_spectral_bandwidth)
features_function.append(extract_features_spectral_contrast)
features_function.append(extract_features_spectral_flatness)
features_function.append(extract_features_spectral_rolloff)
features_function.append(extract_features_poly_features)
features_function.append(extract_features_tonnetz)
features_function.append(extract_features_tempogram)


In [16]:
def extract_features(file_name, features_function):
    data = []
   
    for feature_function in features_function:
        feature = feature_function(file_name)
        if isinstance(feature, np.ndarray):
            data.extend(list(feature))
        else:
            data.append(feature)
    return data

In [17]:
def get_audio_name_class(path):
    audio_name_rec = path.split('/')[-1]
#     print (audio_name)
    audio_name = re.sub(r'\.wav', "", audio_name_rec)
#     print (audio_name_wo_ext)
    label = path.split('/')[-2]
#     print (label)
    return audio_name, label

In [18]:
  def run(dataset_path, pickle_name):
    
    #Captura os nomes das pastas
    name_folders = glob.glob(dataset_path + '*')
    labels = []
    features = []
    
    for names_ in name_folders:

        #Captura o caminho completo do audio
        path_audios = glob.glob(names_ + '/*.wav')
        
        for path_ in tqdm(path_audios):
            
            audio_name, label = get_audio_name_class(path_)
            labels.append(label)
            features.append(extract_features(path_, features_function))
            
    
    pickle.dump(features, open('handcrafted_features_385_' + str(pickle_name) +'.pickle', 'wb'))
    pickle.dump(labels, open('handcrafted_labels_385_' + str(pickle_name) +'.pickle', 'wb'))
    

In [ ]:
run('/home/arlaxad/Documentos/Sistemas de Informação/2020.3/base_de_dados/extendedballroom_v1 (cópia).1(.wav)/Teste/', 'teste')

 33%|███▎      | 142/429 [07:07<14:13,  2.97s/it]

## The one who checks the pickles

In [ ]:
def read_pickle(name):
    with (open(name, 'rb')) as openfile:
        while True:
            try:
                one_instance = pickle.load(openfile)
            except EOFError:
                break
    one_instance = np.asanyarray(one_instance)
    return one_instance

In [ ]:
x_teste = read_pickle('handcrafted_features_teste.pickle')
print("Dados do Teste", x_teste.shape)
y_teste = read_pickle('handcrafted_labels_teste.pickle')
print("Label do Teste", y_teste.shape)
x_treino = read_pickle('handcrafted_features_treino.pickle')
print("Dados do Treino", x_treino.shape)
y_treino = read_pickle('handcrafted_labels_treino.pickle')
print("Label do Treino", y_treino.shape)

# Dados armazenados no Treino

In [ ]:
for id_,i in enumerate(x_treino):
    print(id_,i, "\n")

# Tempo para execulção do algoritmo

| Tempo de treino: ≅ 12 Horas  |Tempo de teste: ≅ 3 horas
|----------|----------|
|![image1](treino.png)|![image2](teste.png)|

### Itens a serem entregues

 - [x] Gerar os pickles de Label e dados do treino
 - [x] Gerar os pickles de Label e dados do teste
 - [x] Mostrar os dados dos pickles

$Atividade (2.5 pt):$ apresentar um notebook gerando os pickles com os vetores de características e respectivas labels. Desse modo, teremos 4 pickle, features e labels de treino e features e labels de teste. 